In [1]:
from collections import defaultdict
import numpy as np
import gurobipy as grb
import pandas as pd

In [3]:
pmax = pd.read_csv('./pgmax.csv')
pmin = pd.read_csv('./pgmin.csv')
ru = pd.read_csv('./ramp.csv')
UT = pd.read_csv('./lu.csv')
DT = pd.read_csv('./ld.csv')    
demand = pd.read_csv('./Demand.csv')   
c_op = pd.read_csv('./cost_op.csv') 
c_st = pd.read_csv('./cost_st.csv') 
PTDF = pd.read_csv('./PTDF.csv') 
busgen = pd.read_csv('./busgen.csv')
busload = pd.read_csv('./busload.csv')
fmax = pd.read_csv('./fmax.csv')


PTDF = pd.DataFrame(PTDF)
busgen = pd.DataFrame(busgen)
busload = pd.DataFrame(busload)
demand = pd.DataFrame(demand)


# Convert the PTDF and demand string elements to a matrix
PTDF_matrix = np.vstack([list(map(float, row[0].split(';'))) for row in PTDF.values])
busgen_matrix = np.vstack([list(map(float, row[0].split(';'))) for row in busgen.values])
busload_matrix = np.vstack([list(map(float, row[0].split(';'))) for row in busload.values])
demand = np.vstack([list(map(float, row[0].split(';'))) for row in demand.values])

Hg = np.dot(PTDF_matrix, busgen_matrix)
Hl = np.dot(PTDF_matrix, busload_matrix)



In [4]:
model = grb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07


In [5]:
n_g=54
n_t=24
n_load=91
n_line=186
# b denotes on/off status and u denotes start start-up status
b = model.addVars(n_g, n_t, vtype=grb.GRB.BINARY)
u = model.addVars(n_g, n_t, vtype=grb.GRB.BINARY)
# how much to produce
p = model.addVars(n_g, n_t, vtype=grb.GRB.CONTINUOUS)
# slack variables
ls = model.addVars(n_t, 1, vtype=grb.GRB.CONTINUOUS)
ws = model.addVars(n_t, 1, vtype=grb.GRB.CONTINUOUS)
model.update()

In [6]:
for t in range(n_t):
    for j in range(n_g):
        # Constraints limiting the production at each generator based on min/max and whether the gen is on.
        model.addConstr(p[j,t] <= pmax.iloc[j].item() *b[j,t])
        model.addConstr(p[j,t] >= pmin.iloc[j].item() *b[j,t])
        if t >= 1:
            # Ramp-up limit:
            model.addConstr(p[j,t]- p[j,t-1] <= ru.iloc[j].item())
            # Ramp-down limit:
            model.addConstr(p[j,t-1]- p[j,t] <= ru.iloc[j].item())
            # u[j,t] == 1 if the generator started in hour t.
            model.addConstr(u[j,t] >= b[j,t]-b[j,t-1])

In [7]:
for t in range(n_t):
    for j in range(n_g):
        if t >= 1:
            # The minimal number of hours each generator should run if started.
            min_on_time = min(t + UT.iloc[j].item() - 1, n_t)
            for tau in range(t, min_on_time):
                model.addConstr(b[j, tau] >= b[j, t] - b[j, t - 1])
            
            # The minimal number of hours each generator should be off if stopped.
            min_off_time = min(t + DT.iloc[j].item() - 1, n_t)
            for tau in range(t, min_off_time):
                model.addConstr(1 - b[j, tau] >= b[j, t - 1] - b[j, t])


In [8]:
for t in range(n_t):
    # The total production during each hour needs to match the total demand.
    model.addConstr(sum(p[j,t] for j in range(n_g)) == sum(demand[t, i] for i in range(n_load)))
model.update()

In [15]:
for t in range(n_t):
    for i in range(n_line):
        # The expression for the total load on each line in the network.
        # Based on the positive flow added from the generators and the negative flow added from the loads.
        expr = sum(Hg[i, j] * p[j, t] for j in range(n_g)) - sum(Hl[i, j] * demand[t,j] for j in range(n_load))
        model.addConstr(expr <= fmax.iloc[i].item())

In [16]:
# The objective is to minimize the cost of power and start-up costs.
expr = sum(c_op.iloc[j].item() * p[j, t] for j in range(n_g) for t in range(n_t)) +\
    sum(c_st.iloc[j].item() * u[j, t] for j in range(n_g) for t in range(n_t))
model.setObjective(sense=grb.GRB.MINIMIZE, expr=expr)
model.update()

In [17]:
opt = model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22958 rows, 3936 columns and 456986 nonzeros
Model fingerprint: 0xaaaf0584
Variable types: 1344 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [5e-07, 4e+02]
  Objective range  [8e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 4e+03]
Presolve removed 10299 rows and 85 columns
Presolve time: 0.25s
Presolved: 12659 rows, 3851 columns, 46555 nonzeros
Variable types: 1313 continuous, 2538 integer (2538 binary)
Found heuristic solution: objective 964004.40700

Root relaxation: objective 8.272913e+05, 1673 iterations, 0.08 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It

In [20]:
for t in range(1):
    for j in range(n_g):
        print(p[j, t].x)

0.0
0.0
0.0
0.0
207.80430000000024
0.0
0.0
0.0
0.0
0.0
350.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
250.0
250.0
0.0
0.0
0.0
0.0
0.0
420.0
420.0
80.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
300.0
200.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
